<a href="https://colab.research.google.com/github/MitchMathieu/cisc452-a3/blob/master/CISC452_a3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CISC 452 - Assignment 3
IMPLEMENT A HYBRID COMPETITIVE LEARNING NETWORK

In [1]:
! git clone https://github.com/MitchMathieu/cisc452-a3.git
! ls cisc452-a3

Cloning into 'cisc452-a3'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.
CISC452_a3.ipynb  iris_test.txt  iris_train.txt


In [0]:
import numpy as np
import random

In [0]:
def read_txt(file_path):
  x = []
  y = []
  f = open(file_path, 'r')
  for line in f:
    line = line.rstrip().split(',')
    x.append(np.array(line[:-1], dtype=np.float32))
    y.append(line[-1])
  return np.array(x), np.array(y)

In [0]:
train_x, train_y = read_txt("cisc452-a3/iris_train.txt")
test_x, test_y = read_txt("cisc452-a3/iris_test.txt")
# print(train_x)
# print(train_y)

Encoding

In [0]:
def cat_to_vector(label):
  # one hot encode the flower type
  if label == "Iris-setosa":
    return np.array([1, 0, 0], dtype=np.int)
  elif label == "Iris-versicolor":
    return np.array([0, 1, 0], dtype=np.int)
  elif label == "Iris-virginica":
    return np.array([0, 0, 1], dtype=np.int)
    
def vector_to_cat(vec):
  if np.array_equal(vec, [1, 0, 0]):
    return "Iris-setosa"
  elif np.array_equal(vec, [0, 1, 0]):
    return "Iris-versicolor"
  elif np.array_equal(vec, [0, 0, 1]):
    return "Iris-virginica"
  
def onehot_iris(arr):
  arr_enc = []
  for cat in arr:
    vec = cat_to_vector(cat)
    arr_enc.append(vec)
  return np.array(arr_enc)

In [0]:
train_y_enc = onehot_iris(train_y)
test_y_enc = onehot_iris(test_y)

# LVQ1

In [0]:
from math import sqrt

In [0]:
class LVQ:
  def __init__(self, num_inputs, num_outputs):
    self.weights = np.random.rand(num_outputs, num_inputs)
    
  def weight_to_class(self, cv):
    arr = np.zeros(3)
    for i, w in enumerate(self.weights):
      if np.array_equal(w, cv):
        arr[i] = 1
    return arr
    
  def euclidian_dist(self, vec1, vec2):
    distance = 0.0
    for i in range(len(vec1)):
      distance += (vec1[i] - vec2[i])**2
    return sqrt(distance)
  
  def find_closest_vec(self, pattern):
    distances = []
    for vec in self.weights:
      dist = self.euclidian_dist(pattern, vec)
      distances.append((vec, dist))
    distances.sort(key=lambda tup: tup[1])
    return distances[0][0]
  
  def train(self, train_x, train_y, lrate, epochs):
    for epoch in range(epochs):
      rate = lrate * (1.0 - (epoch/float(epochs)))
      sum_err = 0.0
      for pattern, true_label in zip(train_x, train_y):
        closest_vec = self.find_closest_vec(pattern)
        pred_label = self.weight_to_class(closest_vec)
        for i in range(pattern.size):
          error = pattern[i] - closest_vec[i]
          sum_err += error**2
          if np.array_equal(pred_label, true_label):
            closest_vec[i] += rate * error
          else:
            closest_vec[i] -= rate * error
      print(f">epoch={epoch+1}, lrate={rate:.3f}, error={sum_err:.3f}")
      
  def test(self, test_x, test_y):
    correct = 0.0
    for pattern, label in zip(test_x, test_y):
      pred = self.weight_to_class(self.find_closest_vec(pattern))
      if np.array_equal(pred, label):
        correct += 1
      print(f"actual={label} got={pred}")
    print(f"correct={correct}")
    print(f"final accuracy={(correct/len(test_y)):.4f}")

In [9]:
lvq1 = LVQ(4, 3)
print(f"init weights:\n{lvq1.weights}")

init weights:
[[0.62597765 0.42866187 0.42888402 0.61753015]
 [0.32463264 0.10908348 0.64579334 0.87159929]
 [0.36243363 0.49436485 0.84790053 0.26994328]]


In [0]:
#Shuffles x, y and onehot array entirely! Very quick!
def shuffle(x,y,onehot):
    indices = [i for i in range(len(x))]
    np.random.shuffle(indices)
    return x[indices],y[indices],onehot[indices]

In [11]:
train_x, train_y, train_y_enc = shuffle(train_x,
                                       train_y,train_y_enc)

lvq1.train(train_x, train_y_enc, 0.1, 10)
print(f"trained weights:\n{lvq1.weights}")

>epoch=1, lrate=0.100, error=6276.219
>epoch=2, lrate=0.090, error=1767.223
>epoch=3, lrate=0.080, error=864.858
>epoch=4, lrate=0.070, error=842.184
>epoch=5, lrate=0.060, error=855.591
>epoch=6, lrate=0.050, error=832.740
>epoch=7, lrate=0.040, error=830.910
>epoch=8, lrate=0.030, error=809.843
>epoch=9, lrate=0.020, error=804.642
>epoch=10, lrate=0.010, error=794.523
trained weights:
[[ 4.87247569  3.83628637  0.53129093 -0.13207495]
 [-2.24274846 -2.17750142  0.04237512  0.6087188 ]
 [ 7.15599401  3.0551152   7.20661416  3.12351043]]


In [12]:
lvq1.test(test_x, test_y_enc)

actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[0 1 0] got=[1. 0. 0.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[1. 0. 0.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[1. 0. 0.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
correct=20.0
final accuracy=0.6667


# PCA - ANN Approach


In [0]:
class PCA:
  def __init__(self, num_inputs, num_outputs):
    self.weights = np.random.rand(num_outputs, num_inputs)
    
  def update_weights(self, pattern, lrate):
    deltas = np.ndarray((len(self.weights), len(self.weights[0])))
    for j in range(len(self.weights)):
      for i in range(len(self.weights[0])):
        nary_sum = 0
        for k in range(j+1):
          nary_sum += np.dot(self.weights[k], pattern) * self.weights[k][i]
        delta = lrate * (np.dot(self.weights[j], pattern)) * (pattern[i] - nary_sum)
        deltas[j][i] = delta
    self.weights += deltas
  
  def train(self, train_x, lrate, iterations):
    # for loop covering number of iterations
    for iteration in range(iterations):
      print(f"\nIteration={iteration+1}")
      # in each iteration, loop through all input patterns
      for pattern in train_x:
        self.update_weights(pattern, lrate)
      print(f"FINAL weights:\n{self.weights}")
    

In [24]:
pca = PCA(4, 3)
print(pca.weights)

[[0.38194598 0.80253738 0.38749277 0.58541297]
 [0.31299597 0.72888351 0.17228877 0.78510612]
 [0.48087847 0.70037853 0.20458419 0.01120433]]


In [25]:
pca.train(train_x, 0.01, 10)


Iteration=1
FINAL weights:
[[ 0.77655226  0.41360993  0.48240851  0.13983704]
 [ 0.05341689 -0.01838327  0.0792373   0.04635522]
 [ 0.31064332  0.25798013 -0.42766083 -0.55486346]]

Iteration=2
FINAL weights:
[[ 0.77655226  0.41360993  0.48240851  0.13983704]
 [-0.10231597 -0.2605531   0.3805412   0.1966484 ]
 [ 0.2852452   0.4638215  -0.62104247 -0.35439279]]

Iteration=3
FINAL weights:
[[ 0.77655226  0.41360993  0.48240851  0.13983704]
 [-0.27095737 -0.51080208  0.68725533  0.34872151]
 [ 0.16399287  0.25863803 -0.33371481 -0.17781623]]

Iteration=4
FINAL weights:
[[ 0.77655226  0.41360993  0.48240851  0.13983704]
 [-0.28051273 -0.51853563  0.69295     0.35097616]
 [ 0.10921941  0.1691346  -0.21717591 -0.11571594]]

Iteration=5
FINAL weights:
[[ 0.77655226  0.41360993  0.48240851  0.13983704]
 [-0.28102048 -0.518695    0.6927932   0.35084427]
 [ 0.08650615  0.1328757  -0.17084121 -0.09132171]]

Iteration=6
FINAL weights:
[[ 0.77655226  0.41360993  0.48240851  0.13983704]
 [-0.281058